# Code Set Up

In [ ]:
pip install neo4j

In [ ]:
# import modules
from neo4j import GraphDatabase
import pandas as pd
from itertools import groupby
import pickle


In [ ]:
# Set up neo4J Connection
# Replace with the actual URI, username, and password
AURA_CONNECTION_URI = "neo4j+s://5a980a50.databases.neo4j.io"
AURA_USERNAME = "neo4j"
AURA_PASSWORD = "Fat4vMlGTgvjEJGwkRl24DE9BYgKeQtYjvUGgo8p2R4"
# Create a driver instance to connect to the database
driver = GraphDatabase.driver(AURA_CONNECTION_URI, auth=(AURA_USERNAME, AURA_PASSWORD))

# Content Based Recommendations System

In [ ]:
FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/postgres/"
df_job_TSC = pd.read_csv(FILE_PATH + 'Job TSC.csv')

In [ ]:
grouped = df_job_TSC.groupby('jobid')

jobs_dict = {}
for job, group in grouped:
    skills_dict = {}
    for index, row in group.iterrows():
        skills_dict[row['TSC Key ID']] = row['Proficiency Level']
    jobs_dict[job] = skills_dict

print(jobs_dict)


{1: {1749.0: 3, 1651.0: 3, 1747.0: 3, 1718.0: 3, 1657.0: 3, 1755.0: 2, 1673.0: 3, 1758.0: 2, 1666.0: 3, 1667.0: 3, 1774.0: 3, 1715.0: 3, 1683.0: 2, 1717.0: 3}, 2: {2613.0: 3, 2588.0: 2, 2589.0: 3, 2592.0: 3, 2614.0: 3, 2712.0: 3, 2616.0: 3, 2643.0: 3, 2617.0: 3, 2618.0: 3, 2733.0: 3, 2601.0: 3, 2602.0: 3, 2605.0: 3, 2641.0: 3, 2647.0: 2}, 3: {708.0: 5, 693.0: 4, 705.0: 5, 679.0: 3, 676.0: 5, 695.0: 5, 716.0: 4, 684.0: 5, 709.0: 5, 706.0: 4, 694.0: 4, 698.0: 6, 710.0: 5, 687.0: 5, 699.0: 5, 688.0: 5, 719.0: 4, 720.0: 4, 700.0: 5, 704.0: 5, 714.0: 5, 727.0: 5, 715.0: 5}, 4: {2740.0: 5, 2596.0: 4, 2597.0: 4, 2643.0: 3, 2678.0: 4, 2733.0: 4, 2660.0: 4, 2687.0: 4, 2743.0: 4, 2744.0: 4, 2745.0: 4, 2746.0: 4, 2747.0: 4, 2748.0: 4, 2604.0: 4, 2695.0: 3, 2708.0: 4, 2710.0: 5, 2711.0: 4, 2732.0: 4, 2684.0: 3, 2720.0: 3, 2642.0: 4, 2647.0: 3}, 5: {1943.0: 4, 1944.0: 4, 1947.0: 5, 1898.0: 4, 1936.0: 4, 1937.0: 4, 1948.0: 4, 1923.0: 5, 1965.0: 5, 1902.0: 4, 1951.0: 4, 1904.0: 5, 1905.0: 4, 1906.0: 

In [ ]:
# export data into pickle file to avoid repeated reading in and recalculation
filename = 'content_jobs_data.pkl'

# Writing jobs_dict to the pickle file
with open(filename, 'wb') as file:
    pickle.dump(jobs_dict, file)

In [ ]:
import neo4j
def load_query(query):
  records, summary, keys  = driver.execute_query(
      query,
      database_="neo4j",
  )
  return records, summary, keys

In [ ]:
# Getting similar skills job r/s based on PTE
def fetch_data(query):
    with driver.session() as session:
        return list(session.run(query))

# Extract user and job skills from Neo4j
query = """
// Obtained Learner Profile Information
MATCH (u:LearnerProfile {uuid: "2023-09-18T03:21:21.024Z-meow@gmail.com"})

// Gather all job nodes associated with the learner
OPTIONAL MATCH (u)-[:HAS_PREV_JOB|:HAS_CUR_JOB]->(j:Job)
WITH u, COLLECT(DISTINCT j.jobid) as jobids

// Traverse each job to get the skills
UNWIND jobids AS jobid
OPTIONAL MATCH (job:Job {jobid: jobid})-[:REQUIRE_TECHNICAL_SKILL]->(jt:JobTSC)-[:HAS_TSC_KEY]->(k:TSCKey)-[:HAS_PROFICIENCY]->(p:TSCProficiency)

// Direct technical skills of the user
OPTIONAL MATCH (u)-[:HAS_TECHNICAL_SKILLS]->(s:TSCProficiency)

WITH u, jobids,
     REDUCE(output = [], skill IN COLLECT(DISTINCT {keyID: s.`TSC Key ID`, proficiency: s.`Proficiency Level`}) | output + skill) +
     REDUCE(output = [], skill IN COLLECT(DISTINCT {keyID: p.`TSC Key ID`, proficiency: p.`Proficiency Level`}) | output + skill) AS AllSkills
RETURN AllSkills as data, jobids
"""
user_skills_data = fetch_data(query)

driver.close()

In [ ]:
user_skills = user_skills_data[0]['data']

In [ ]:
exclude_jobs = user_skills_data[0]['jobids']
exclude_jobs

[1461, 1082]

In [ ]:
# Filter out the unwanted job IDs when constructing the matrix
jobs_dict = {key: val for key, val in jobs_dict.items() if key not in exclude_jobs}


In [ ]:
user_skills.sort(key=lambda x: x['keyID'])
filtered_user_skills = [next(g) for k, g in groupby(user_skills, key=lambda x: x['keyID'])]
user_skills_dict = {skill['keyID']: skill['proficiency'] for skill in filtered_user_skills}
# TODO: what happends if user has multiple levels of proficiency

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Convert dictionaries to vectors for calculation
all_skills = set(user_skills_dict.keys()) | set(s for job_skills in jobs_dict.values() for s in job_skills.keys())
user_vector = [user_skills_dict.get(skill, 0) for skill in all_skills]
jobs_matrix = [[job_skills.get(skill, 0) for skill in all_skills] for job_skills in jobs_dict.values()]

# Compute similarity scores
similarities = cosine_similarity([user_vector], jobs_matrix)[0]

# Get the top 10 job IDs based on similarity scores
top_10_indices = np.argsort(similarities)[-10:][::-1]  # -10: gets the last 10 values, [::-1] reverses the array
top_10_job_ids = [list(jobs_dict.keys())[i] for i in top_10_indices]

print(top_10_job_ids)


[1251, 1320, 158, 993, 1383, 1592, 1451, 427, 868, 566]


# Collaborative Filtering


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import OrderedDict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Fake data set of user histories generated with chat gpt
userProfiles = [
    {"userID": 1, "pastJobs": [703, 1], "currentJob": 1065},
    {"userID": 2, "pastJobs": [1614], "currentJob": 611},
    {"userID": 3, "pastJobs": [1135], "currentJob": 1412},
    {"userID": 4, "pastJobs": [572], "currentJob": 1396},
    {"userID": 5, "pastJobs": [957, 1384], "currentJob": 1597},
    {"userID": 6, "pastJobs": [1314], "currentJob": 1174},
    {"userID": 7, "pastJobs": [807], "currentJob": 1429},
    {"userID": 8, "pastJobs": [955, 633], "currentJob": 905},
    {"userID": 9, "pastJobs": [835], "currentJob": 1595},
    {"userID": 10, "pastJobs": [1729], "currentJob": 397},
    #From Business Analyst -> Product Manager -> Senior Product Manager
    {"userID": 1, "pastJobs": [143, 1314], "currentJob": 1174},

    #From Associate Software Engineer -> Software Engineer -> Software Architect
    {"userID": 2, "pastJobs": [1614, 611], "currentJob": 741},

    #From IT Auditor -> IT Audit Manager -> Head of IT Audit
    {"userID": 3, "pastJobs": [1135, 1412], "currentJob": 844},

    # From Associate Business Analyst -> Business Intelligence Manager -> Business Intelligence Director
    {"userID": 4, "pastJobs": [703, 1541], "currentJob": 1739},

    # Data progression: Associate Data Engineer -> Data Engineer -> Senior Data Engineer
    {"userID": 5, "pastJobs": [807, 309], "currentJob": 524},

    #Security progression: Associate Security Analyst -> Security Operations Analyst -> Security Operations Manager
    {"userID": 6, "pastJobs": [835, 1595], "currentJob": 1486},

    #From Associate UX Designer -> UX Designer -> Lead UX Designer
    {"userID": 7, "pastJobs": [1, 1065], "currentJob": 1657},

    #Sales progression: Sales Executive -> Sales Account Manager -> Channel Sales Manager
    {"userID": 8, "pastJobs": [751, 230], "currentJob": 1402},

    #Infrastructure progression: Associate Infrastructure Engineer -> Infrastructure Engineer -> Head of Infrastructure
    {"userID": 9, "pastJobs": [348, 1597], "currentJob": 1465},

    #Leadership progression: Program Manager -> Program Director -> Chief Technology Officer
    {"userID": 10, "pastJobs": [456, 1056], "currentJob": 1367},

    #Data science progression: Data Scientist -> Head of Data Science and AI -> Chief Data Officer/Chief AI Officer
    {"userID": 11, "pastJobs": [1429, 863], "currentJob": 1512},

    #Network progression: Associate Network Engineer -> Network Engineer -> Infrastructure Architect
    {"userID": 12, "pastJobs": [1384, 957], "currentJob": 1703},

    #Quality progression: Quality Engineer -> Quality Engineering Manager -> Head of Quality
    {"userID": 13, "pastJobs": [1607, 668], "currentJob": 1290},

    #Marketing progression: Marketing Executive -> Marketing Manager -> Marketing Director
    {"userID": 14, "pastJobs": [1308, 955], "currentJob": 633},

    #Security progression: Forensics Investigator -> Incident Investigator -> Forensics Investigation Manager
    {"userID": 15, "pastJobs": [155, 968], "currentJob": 649},

    #Architectural Progression:
    {"userID": 16, "pastJobs": [611, 741], "currentJob": 1584},
    {"userID": 17, "pastJobs": [1597, 1703], "currentJob": 1465},

    #AI and Data Science Evolution:
    {"userID": 18, "pastJobs": [1560, 944, 1429], "currentJob": 863},

    #Cybersecurity Progression:
    {"userID": 19, "pastJobs": [835, 582], "currentJob": 1317},
    {"userID": 20, "pastJobs": [1317], "currentJob": 1029},

    # Sales and Customer Interaction:
    {"userID": 21, "pastJobs": [1656, 230], "currentJob": 1402},
    {"userID": 22, "pastJobs": [1402], "currentJob": 130},

    #Quality and Assurance:
    {"userID": 23, "pastJobs": [472], "currentJob": 1329},
    {"userID": 24, "pastJobs": [1329], "currentJob": 1290},

    #Database and Data Engineering:
    {"userID": 25, "pastJobs": [1197, 1464], "currentJob": 309},
    {"userID": 26, "pastJobs": [309], "currentJob": 524},

    # Operational Roles:
    {"userID": 27, "pastJobs": [1058, 144], "currentJob": 681},

    # Product Management and Leadership:
    {"userID": 28, "pastJobs": [703, 1314], "currentJob": 420},
    {"userID": 29, "pastJobs": [420], "currentJob": 1367},

    # Networking and Communication:
    {"userID": 30, "pastJobs": [1163, 1477], "currentJob": 816},

    # Embedded Systems Progression:
    {"userID": 31, "pastJobs": [1337], "currentJob": 997},
    {"userID": 32, "pastJobs": [997], "currentJob": 377},

    # Generating slightly similar user profile so that collaborative algorithm could work
    # Similar Data Science Progression with variations:
    {"userID": 33, "pastJobs": [1560, 1429], "currentJob": 863},
    {"userID": 34, "pastJobs": [944, 1560], "currentJob": 1429},

    # Slightly varied security progression:
    {"userID": 35, "pastJobs": [582], "currentJob": 1317},
    {"userID": 36, "pastJobs": [1317, 1029], "currentJob": 1595},

    # Sales with a touch of marketing:
    {"userID": 37, "pastJobs": [1656, 230, 1308], "currentJob": 955},
    {"userID": 38, "pastJobs": [955, 1402], "currentJob": 633},

    # Quality Assurance with leadership progression:
    {"userID": 39, "pastJobs": [472, 1329], "currentJob": 668},
    {"userID": 40, "pastJobs": [1329, 668], "currentJob": 1290},

    # Diverse data engineering:
    {"userID": 41, "pastJobs": [1464, 1197], "currentJob": 309},
    {"userID": 42, "pastJobs": [309, 524], "currentJob": 807},

    # Operations with variations:
    {"userID": 43, "pastJobs": [1058], "currentJob": 681},
    {"userID": 44, "pastJobs": [681, 144], "currentJob": 1174},

    # Product and UX Design:
    {"userID": 45, "pastJobs": [703, 1], "currentJob": 1065},
    {"userID": 46, "pastJobs": [1065, 420], "currentJob": 1174},

    # Networking with Infrastructure progression:
    {"userID": 47, "pastJobs": [1163, 957], "currentJob": 816},
    {"userID": 48, "pastJobs": [1477, 1384], "currentJob": 1597},

    # Embedded and Systems:
    {"userID": 49, "pastJobs": [1337, 997], "currentJob": 377},
    {"userID": 50, "pastJobs": [997, 377], "currentJob": 1337},

    # Data to AI:
    {"userID": 51, "pastJobs": [1560, 309], "currentJob": 324},
    {"userID": 52, "pastJobs": [944, 324], "currentJob": 1429},

    # UX to Product Management:
    {"userID": 53, "pastJobs": [1, 1065], "currentJob": 420},
    {"userID": 54, "pastJobs": [1065, 1174], "currentJob": 1314},

    # Operational to Strategy:
    {"userID": 55, "pastJobs": [1058, 681], "currentJob": 1174},
    {"userID": 56, "pastJobs": [144, 681], "currentJob": 1174},

    # Networking with variations:
    {"userID": 57, "pastJobs": [1163, 957], "currentJob": 1703},
    {"userID": 58, "pastJobs": [1477, 1703], "currentJob": 816},

    # Embedded variations:
    {"userID": 59, "pastJobs": [1337], "currentJob": 377},
    {"userID": 60, "pastJobs": [377, 997], "currentJob": 1337},

    # Unconventional Career Path
     #From technical to sales/marketing:
    {"userID": 33, "pastJobs": [611], "currentJob": 230},  # Software Engineer -> Sales Account Manager
    {"userID": 34, "pastJobs": [1429], "currentJob": 1308},  # Data Scientist -> Marketing Executive

    # From operational to strategic:
    {"userID": 35, "pastJobs": [144], "currentJob": 1174},  # Operations and Support Manager -> Senior Product Manager

    # From cybersecurity to AI:
    {"userID": 36, "pastJobs": [1595], "currentJob": 324},  #Security Operations Analyst -> AI/ML Engineer

    # From sales to AI/data:
    {"userID": 37, "pastJobs": [751], "currentJob": 1429},  #Sales Executive -> Data Scientist

    # From infrastructure to design:
    {"userID": 38, "pastJobs": [816], "currentJob": 1},  #Infrastructure Engineering Manager -> Associate UX Designer

    # From audit to engineering:
    {"userID": 39, "pastJobs": [1412], "currentJob": 611},  # Audit Manager -> Software Engineer

    # From product management to cybersecurity:
    {"userID": 40, "pastJobs": [420], "currentJob": 1029},  #Head of Product -> Chief Information Security Officer

    # From AI Research to operations:
    {"userID": 41, "pastJobs": [469], "currentJob": 144},  # AI Applied Researcher -> Operations and Support Manager

    # From sales to quality assurance:
    {"userID": 42, "pastJobs": [1402], "currentJob": 1607},  # Channel Sales Manager -> Quality Engineer

    # From business intelligence to networking:
    {"userID": 43, "pastJobs": [1739], "currentJob": 957},  #Business Intelligence Director -> Network Engineer

    # From DevOps to sales:
    {"userID": 44, "pastJobs": [478], "currentJob": 751},  # DevOps Engineer -> Sales Executive

    # From head roles to engineering:
    {"userID": 45, "pastJobs": [106], "currentJob": 611},  # Head of Software Engineering -> Software Engineer

    # From UX design to IT audit:
    {"userID": 46, "pastJobs": [1065], "currentJob": 1135},  # UX Designer -> IT Auditor

    # From database support to business development:
    {"userID": 47, "pastJobs": [1197], "currentJob": 1303},  # Associate Database Support Engineer -> Business Development Manager

]


In [ ]:
FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/postgres/"
df_collab_recc = pd.read_csv(FILE_PATH + 'Collaborative Filtering.csv')
df_skills = pd.read_csv(FILE_PATH + 'Skills Collaborative Filtering.csv')
df_jobs = pd.read_csv(FILE_PATH + 'Jobs Collaborative Filtering.csv')

In [ ]:
# Combine past and current jobs
userJobsList = list(map(lambda x: x['pastJobs'] + [x['currentJob']], userProfiles))

#If you're aiming for a broader set of recommendations that take into account a user's entire career path, then treating all jobs—past and current—equally might provide a more diverse set of job recommendations.
# Function to get TSC ID and proficiency level
# Function to get TSC ID and proficiency level
def extract_info(jobids):
    filtered_df = df_collab_recc[df_collab_recc['jobid'].isin(jobids)]
    info_dict = dict(zip(filtered_df['TSC Key ID'], filtered_df['Proficiency Level']))
    return [jobids, info_dict]

# Get results for each sublist of jobids
user_hist_list = [extract_info(ids) for ids in userJobsList]

# Create a DataFrame from the user history
user_df = pd.DataFrame(user_hist_list, columns=['Jobs', 'Skills'])

scaler = StandardScaler()
df_skills['Z-Normalized'] = df_skills.groupby('TSC Key ID')['Proficiency Level'].transform(lambda x: scaler.fit_transform(x.values.reshape(-1,1)).squeeze())


In [ ]:
# Assuming user_df is your user data DataFrame and df_jobs is your jobs data DataFrame

# Create a user-item matrix for skills
num_users = len(user_df)
all_skills = df_skills['TSC Key ID'].unique()
#skills_index_map = {skill: idx for idx, skill in enumerate(all_skills)} # Not important
skills_matrix = pd.DataFrame(0, index=range(num_users), columns=all_skills)

# Create a user-item matrix for jobs using df_jobs
all_jobs = df_jobs['jobid'].unique()
jobs_index_map = {idx: job for idx, job in enumerate(all_jobs)} # maps index to jobs
jobs_matrix = pd.DataFrame(0, index=range(num_users), columns=all_jobs)

# Fill the skills and jobs matrices using the user's history
for idx, row in user_df.iterrows():
    user_skills = row['Skills']
    user_jobs = row['Jobs']

    # Fill skills matrix
    for skill, proficiency in user_skills.items():
        # Using Z-normalized proficiency levels here, but you can adjust as needed
        skills_matrix.at[idx, skill] = df_skills[(df_skills['TSC Key ID'] == skill) & (df_skills['Proficiency Level'] == proficiency)]['Z-Normalized'].values[0]

    # Fill jobs matrix
    for job in user_jobs:
        jobs_matrix.at[idx, job] = 1

# Concatenate the skills and jobs matrices
combined_matrix = pd.concat([jobs_matrix, skills_matrix], axis=1)

combined_matrix = combined_matrix.values


In [ ]:
current_user_row = combined_matrix[0]
current_user_row.shape

(238,)

In [ ]:
# Define the current user's row (This will be replaced by the user history from the application)
current_user_row = combined_matrix[0]

# Calculate cosine similarity between the current user and all other users
user_similarity_scores = cosine_similarity([current_user_row], combined_matrix)

# Sort users by similarity
similar_users = np.argsort(user_similarity_scores[0])[::-1][1:] # excludes the first user

# Initialize an OrderedDict to collect recommended job indices
recommended_job_indices = OrderedDict()

# Recommend jobs based on similar users' history
for user in similar_users:
    # Find jobs that the similar user has interacted with
    similar_user_jobs = combined_matrix[user, : len(all_jobs)+1]  # Include only job columns
    # Filter jobs that the current user hasn't interacted with
    new_jobs_indices = np.where((similar_user_jobs > 0))
    # Update the OrderedDict with new job indices
    for idx in new_jobs_indices[0]:
        recommended_job_indices[idx] = None
    # You can break after collecting a certain number of recommendations, e.g., 5
    if len(recommended_job_indices) >= 5:
        break

# Convert the OrderedDict keys to a list of recommended job indices
recommended_job_indices = list(recommended_job_indices.keys())

# Map the indices directly to job IDs using the switched jobs_index_map
recommended_job_ids = [jobs_index_map[idx] for idx in recommended_job_indices]

# Now, you have a list of recommended job IDs
print(recommended_job_ids)

[1, 703, 1065, 1135, 1657]


In [ ]:
# Store trained user history into a pickle
import pickle

# Create a dictionary to store both the combined_matrix and jobs_index_map
data_to_store = {
    'combined_matrix': combined_matrix,
    'jobs_index_map': jobs_index_map
}

# Define the filename for the pickle file
pickle_filename = 'recommendation_data.pkl'

# Save the data to the pickle file
with open(pickle_filename, 'wb') as file:
    pickle.dump(data_to_store, file)

print(f"Data saved to {pickle_filename}")


Data saved to recommendation_data.pkl


In [ ]:
import pickle

# Load the data from the pickle file
with open('recommendation_data.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

# Access the combined_matrix and jobs_index_map
combined_matrix = loaded_data['combined_matrix']
jobs_index_map = loaded_data['jobs_index_map']

# Now, you can use combined_matrix and jobs_index_map for recommendations with new user data


# Heuristics Job Reccomendations


In [ ]:
FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/postgres/"
df_jobs = pd.read_csv(FILE_PATH + 'Jobs.csv')

In [ ]:
df_jobs

,Sector,Track,Job Role,jobid
0,Infocomm Technology,Strategy and Governance,Associate UX Designer,1
1,Sea Transport,Maritime Services,Assistant Chartering Broker / Trainee Charteri...,2
2,Electronics,Technical and Engineering,Senior Product Engineer,3
3,Sea Transport,Shipping,HSSEQ Superintendent,4
4,Logistics,Sales and Customer Service,Business Development Manager / Sales and Marke...,5
...,...,...,...,...
1737,Media,Media Technology and Operations,Manager - On-Demand Media Technology and Opera...,1738
1738,Infocomm Technology,Data and Artificial Intelligence,Business Intelligence Director,1739
1739,Energy and Power,Gas Systems Operations,Gas Systems Operations Senior Technical Officer,1740
1740,Security,Auxiliary Police,Auxiliary Police Officer,1741


# Embedding based methods (BERT)
Text similarity with Contextual Embedding (BERT)
https://arxiv.org/pdf/2107.00221.pdf

In [ ]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/postgres/"
df_job_descrip = pd.read_csv(FILE_PATH + 'Job Description.csv')
df_job_descrip
df_jobs = pd.read_csv(FILE_PATH + 'Jobs.csv')

In [ ]:
# Join job table with job dec

# Merge dataframes on 'jobid'
merged_df = df_job_descrip.merge(df_jobs, on='jobid', how='inner')

In [ ]:
df_tasks = pd.read_csv(FILE_PATH + 'Job Tasks.csv')
# Group by jobid and join the tasks into a single string
df_combined_tasks = df_tasks.groupby('jobid')['Key Tasks'].apply(' '.join).reset_index()

# Rename columns for clarity
df_combined_tasks.rename(columns={'Key Tasks': 'combined_tasks'}, inplace=True)

print(df_combined_tasks.head())


   jobid                                     combined_tasks
0      1  Implement application site improvements to fun...
1      2  Ensure vessels contracted are in compliance wi...
2      3  Conduct qualification and validation tests for...
3      4  Conduct regular emergency response drills and/...
4      5  Develop intervention strategies to mitigate ga...


In [ ]:
# Merge dataframes on 'jobid'
merged_df = df_combined_tasks.merge(merged_df, on='jobid', how='inner')
merged_df

,jobid,combined_tasks,Job Role Description,Performance Expectations,jdid,Sector,Track,Job Role
0,1,Implement application site improvements to fun...,The Associate User Experience Designer assists...,NaN,1064,Infocomm Technology,Strategy and Governance,Associate UX Designer
1,2,Ensure vessels contracted are in compliance wi...,The Assistant Chartering Broker/Trainee Charte...,NaN,1437,Sea Transport,Maritime Services,Assistant Chartering Broker / Trainee Charteri...
2,3,Conduct qualification and validation tests for...,The Senior Product Engineer generates test set...,NaN,401,Electronics,Technical and Engineering,Senior Product Engineer
3,4,Conduct regular emergency response drills and/...,"The Health, Safety, Security, Environment and ...",In accordance to:\n· International Safet...,1491,Sea Transport,Shipping,HSSEQ Superintendent
4,5,Develop intervention strategies to mitigate ga...,The Business Development Manager/Sales and Mar...,NaN,1139,Logistics,Sales and Customer Service,Business Development Manager / Sales and Marke...
...,...,...,...,...,...,...,...,...
1737,1738,Prepare operational budget for on-demand media...,The Manager - On-Demand Media Technology and O...,NaN,1275,Media,Media Technology and Operations,Manager - On-Demand Media Technology and Opera...
1738,1739,Define the structure and tools to be applied i...,The Business Intelligence Director sets the st...,NaN,1003,Infocomm Technology,Data and Artificial Intelligence,Business Intelligence Director
1739,1740,Prepare technical and product specifications f...,The Gas Systems Operations Senior Technical Of...,NaN,518,Energy and Power,Gas Systems Operations,Gas Systems Operations Senior Technical Officer
1740,1741,Perform crowd control and traffic management d...,The Auxiliary Police Officer is responsible fo...,NaN,1515,Security,Auxiliary Police,Auxiliary Police Officer


In [ ]:
merged_df = merged_df[["Job Role Description", "combined_tasks", "jobid", "Sector", "Job Role"]]

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import pickle
# Load BERT tokenizer and model
# Bert is a contextual algorithm it doesnot require Text Preprocessing and Lemma
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to compute embeddings using BERT
def compute_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        output = model(**inputs)
    # Using mean of the last layer as embedding. Also converting the tensor to numpy.
    return output.last_hidden_state.mean(dim=1).numpy()

# Define the weights for each embedding
weight_combined = 0.5 # Weight for combined job name and sector
weight_description = 0.25
weight_tasks = 0.25

# Function to compute weighted embeddings
def compute_weighted_embedding(row):
    # Combine job name and sector into a single text
    combined_text = f"{row['Job Role']} {row['Sector']}"

    # Compute embeddings for the combined text, description, and tasks
    combined_embedding = compute_embedding(combined_text)
    description_embedding = compute_embedding(row['Job Role Description'])
    tasks_embedding = compute_embedding(row['combined_tasks'])

    # Apply weights and compute the weighted embedding
    weighted_embedding = (
        weight_combined * combined_embedding +
        weight_description * description_embedding +
        weight_tasks * tasks_embedding
    )

    return weighted_embedding

# Initialize an empty DataFrame to store results
result_df = pd.DataFrame(columns=['jobid', 'weighted_embedding'])

# Process data in chunks (e.g., every 100 rows)
chunk_size = 50
for i in range(0, len(merged_df), chunk_size):
    chunk = merged_df[i:i + chunk_size]
    # Apply the compute_weighted_embedding function to each row in the chunk
    chunk['weighted_embedding'] = chunk.apply(compute_weighted_embedding, axis=1)
    # Append the chunk to the result DataFrame
    result_df = result_df.append(chunk[['jobid', 'weighted_embedding']])
    # Save intermediate results
    result_df.to_csv('intermediate_results.csv', index=False)

# Save the final result DataFrame
result_df.to_csv('final_results.csv', index=False)
result_df.to_pickle("embedding_recommendation.pkl")

# Display the jobid and the computed weighted embeddings
print(result_df[['jobid', 'weighted_embedding']])


<ipython-input-16-a597cbbc08ec>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['weighted_embedding'] = chunk.apply(compute_weighted_embedding, axis=1)
<ipython-input-16-a597cbbc08ec>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(chunk[['jobid', 'weighted_embedding']])
<ipython-input-16-a597cbbc08ec>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['weighted_embedding'] = 

     jobid                                 weighted_embedding
0        1  [[-0.09483701, 0.2488901, 0.42554444, 0.072270...
1        2  [[0.18758048, 0.06588058, 0.33755696, 0.075638...
2        3  [[0.115463644, 0.3169006, 0.09703343, -0.08607...
3        4  [[-0.10500783, 0.13360843, 0.29949683, -0.0551...
4        5  [[-0.2876013, 0.16736615, 0.43936235, -0.04973...
...    ...                                                ...
1737  1738  [[-0.10097741, 0.22962189, 0.3461032, -0.03287...
1738  1739  [[-0.2201072, 0.10718223, 0.31572354, 0.028923...
1739  1740  [[-0.04395651, 0.079158776, 0.1839367, 0.02556...
1740  1741  [[-0.057630695, -0.059652314, 0.18723401, -0.2...
1741  1742  [[-0.10497731, 0.29823118, 0.23772895, -0.0003...

[1742 rows x 2 columns]


### Recommendation sections

In [ ]:
import pandas as pd

# Load the DataFrame from the pickle file
result_df = pd.read_pickle("embedding_recommendation.pkl")

In [ ]:
result_df

,jobid,weighted_embedding
0,1,"[[-0.09483701, 0.2488901, 0.42554444, 0.072270..."
1,2,"[[0.18758048, 0.06588058, 0.33755696, 0.075638..."
2,3,"[[0.115463644, 0.3169006, 0.09703343, -0.08607..."
3,4,"[[-0.10500783, 0.13360843, 0.29949683, -0.0551..."
4,5,"[[-0.2876013, 0.16736615, 0.43936235, -0.04973..."
...,...,...
1737,1738,"[[-0.10097741, 0.22962189, 0.3461032, -0.03287..."
1738,1739,"[[-0.2201072, 0.10718223, 0.31572354, 0.028923..."
1739,1740,"[[-0.04395651, 0.079158776, 0.1839367, 0.02556..."
1740,1741,"[[-0.057630695, -0.059652314, 0.18723401, -0.2..."


## Recommendations from user input


In [ ]:
user_job_ids = [1, 1065]
user_embeddings_list = result_df[result_df['jobid'].isin(user_job_ids)]['weighted_embedding'].tolist()

similarities = [
    {
        'JobID': row['jobid'],
        'Similarity': round(np.mean([cosine_similarity(user_embedding.reshape(1, -1), row['weighted_embedding'].reshape(1, -1))[0][0] for user_embedding in user_embeddings_list]), 3)
    }
    for index, row in result_df.iterrows()
]

# Sorting the similarities
sorted_similarities = sorted(similarities, key=lambda x: x['Similarity'], reverse=True)


In [ ]:
sorted_similarities

[{'JobID': 1, 'Similarity': 0.986},
 {'JobID': 1065, 'Similarity': 0.986},
 {'JobID': 1657, 'Similarity': 0.977},
 {'JobID': 1729, 'Similarity': 0.947},
 {'JobID': 741, 'Similarity': 0.944},
 {'JobID': 763, 'Similarity': 0.944},
 {'JobID': 397, 'Similarity': 0.941},
 {'JobID': 828, 'Similarity': 0.94},
 {'JobID': 237, 'Similarity': 0.936},
 {'JobID': 1314, 'Similarity': 0.936},
 {'JobID': 1584, 'Similarity': 0.936},
 {'JobID': 478, 'Similarity': 0.932},
 {'JobID': 1308, 'Similarity': 0.932},
 {'JobID': 1538, 'Similarity': 0.932},
 {'JobID': 1407, 'Similarity': 0.93},
 {'JobID': 1607, 'Similarity': 0.93},
 {'JobID': 377, 'Similarity': 0.929},
 {'JobID': 955, 'Similarity': 0.929},
 {'JobID': 1614, 'Similarity': 0.929},
 {'JobID': 1703, 'Similarity': 0.929},
 {'JobID': 121, 'Similarity': 0.928},
 {'JobID': 611, 'Similarity': 0.927},
 {'JobID': 668, 'Similarity': 0.927},
 {'JobID': 1193, 'Similarity': 0.927},
 {'JobID': 1666, 'Similarity': 0.927},
 {'JobID': 657, 'Similarity': 0.926},
 {'J

In [ ]:

# Get the top 10 items
top_10_similarities = sorted_similarities[:10]